# MPI - operacje punktowe (point to point)

Stosując inferface `mpi4py` 

In [4]:
import os
import tempfile
import execnet
wd = tempfile.mkdtemp()
os.chdir(wd)
os.getcwd()
rsync = execnet.RSync(wd)
group= [execnet.makegateway('ssh=fermi')]


In [5]:
%%writefile $wd/mpi001.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print "OK, rank= ",rank,size

Writing /tmp/tmpdxQ6jV/mpi001.py


In [6]:
for gw in group:
    rsync.add_target(gw, wd)
    rsync.send()

<SshGateway id='gw1' receive-live, 1 active channels> <= mpi001.py
<SshGateway id='gw2' receive-live, 1 active channels> <= mpi001.py


In [8]:
!mpirun -H stratus,fermi,nimbus python mpi001.py



OK, rank=  1 3
OK, rank=  2 3
OK, rank=  0 3


In [12]:
%%writefile mpi001.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
   data = {'a': 7, 'b': 3.14}
   comm.send(data, dest=1, tag=11)
   
elif rank == 1:
   data = comm.recv(source=0, tag=11)
   print "OK, rank= ",rank,"dane: ",data

Overwriting mpi001.py


In [13]:
!mpirun -n 2 python mpi001.py

OK, rank=  1 dane:  {'a': 7, 'b': 3.14}


In [38]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.send(a[0,:], dest=1)
   
elif rank == 1:
   a[0,:] = comm.recv(source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [39]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [40]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.send(a[:,0], dest=1)
   
elif rank == 1:
   a[:,0] = comm.recv(source=0)
   print "OK,",np.sum(a)

Overwriting mpi001.py


In [41]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [42]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.Send(a[0,:], dest=1)
   
elif rank == 1:
   comm.Recv(a[0,:], source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [43]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [44]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.Send(a[:,0], dest=1)
   
elif rank == 1:
   comm.Recv(a[:,0], source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [45]:
!mpirun -n 2 python mpi001.py

Traceback (most recent call last):
  File "mpi001.py", line 13, in <module>
    comm.Recv(a[:,0], source=0)
  File "Comm.pyx", line 143, in mpi4py.MPI.Comm.Recv (src/mpi4py.MPI.c:61865)
Traceback (most recent call last):
  File "mpi001.py", line 10, in <module>
    comm.Send(a[:,0], dest=1)
  File "message.pxi", line 323, in mpi4py.MPI.message_p2p_recv (src/mpi4py.MPI.c:22363)
  File "Comm.pyx", line 132, in mpi4py.MPI.Comm.Send (src/mpi4py.MPI.c:61677)
  File "message.pxi", line 318, in mpi4py.MPI.message_p2p_send (src/mpi4py.MPI.c:22297)
  File "message.pxi", line 309, in mpi4py.MPI._p_msg_p2p.for_recv (src/mpi4py.MPI.c:22221)
  File "message.pxi", line 301, in mpi4py.MPI._p_msg_p2p.for_send (src/mpi4py.MPI.c:22160)
  File "message.pxi", line 111, in mpi4py.MPI.message_simple (src/mpi4py.MPI.c:20121)
  File "message.pxi", line 111, in mpi4py.MPI.message_simple (src/mpi4py.MPI.c:20121)
  File "message.pxi", line 51, in mpi4py.MPI.message_basic (src/mpi4py.MPI.c:19298)
  File "message.

In [76]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   buf = a[:,0].copy()
   comm.Send(buf, dest=1)
   
elif rank == 1:
   buf = np.empty(2)
   comm.Recv(buf, source=0)
   a[:,0] = buf 
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [77]:
!mpirun -n 2 python mpi001.py

OK, 4.0
